In [1]:
# Dependencies and Setup
import requests
import gmaps
import pandas as pd

# Import API key
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
hotel_df = pd.read_csv("weather_data/hotels.csv")

hotel_df.count()

City_ID       67
City          67
Country       67
Max Temp      67
Lat           67
Lng           67
Hotel Name     0
dtype: int64

In [3]:
# Set the parameters to search for a hotel in Paris.
params = {"radius": 5000,
          "types": "lodging",
          "key": g_key,
          "location": "48.8566, 2.3522"}

# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

hotels

{'html_attributions': [],
 'next_page_token': 'Aap_uEA9evxhYcy5N-SOA2_9uIjYAeZglw_nceWoImGKFYPAPnbLXoGIdcQfZHVT0sfefxyJ7eJwTOH7ZAJbg3yy3WzqZ6iVW45tynbQ2dPnJV9M1_s6vyJanbSp4TimKQMOmo-nrnR_afv5kVtSotrPUsR5CxjmybPiTsYcPdLQcnV2yLiYHjzc9FO37lF39qTwXyTGc5-OusYT-qgMRmKoUV7x9JB7felTyY7L6ZIx1zK2AC0-xAAP2CSr387bVUKypUv3NtJS7-JczD1Qvw3dmk4orJbbwhR-bUQTNX6HcxTwY5BXRJPt6WMiMQ_t-AJzU9fkENTIZmPT7KdxNrkvgMn3OxerTxAGLYOrUtBrR6TltTbq1_v23YpxYq7TBsTn5Ng7azWcWUey2d7ToZ4C4Rk-G-_JWf418QdT6ErfWrpKmKbW0KmSrWsc',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 48.8581126, 'lng': 2.3529277},
    'viewport': {'northeast': {'lat': 48.8594360302915,
      'lng': 2.354355480291502},
     'southwest': {'lat': 48.8567380697085, 'lng': 2.351657519708498}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'icon_background_color': '#909CE1',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
   'nam

In [4]:
len(hotels["results"])

20

In [5]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [6]:
hotel_df.head()

,City_ID,City,Country,Max Temp,Lat,Lng,Hotel Name
0,10,Tazovskiy,RU,18.41,67.4667,78.7000,TazHostel
1,21,Dikson,RU,13.10,73.5069,80.5464,NaN
2,29,Lyuban,BY,36.55,52.7985,28.0048,"Gostinitsa ""Oressa"""
3,32,Svetlyy,RU,39.25,54.6750,20.1347,"Gostinitsa-Kafe, Alena"
4,33,Paamiut,GL,29.03,61.9940,-49.6678,Pistut


In [7]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=max_temp, 
                                 dissipating=False,
                                 max_intensity=300, 
                                 point_radius=4)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=max_temp,
                                 dissipating=False, 
                                 max_intensity=300, 
                                 point_radius=4)

marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [10]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [11]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=max_temp,
                                 dissipating=False,
                                 max_intensity=300, 
                                 point_radius=4)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))